# Disaster Response ETL Pipeline
> Perform ETL on categorized message data.

The goal of the disaster response project is to process and flag (categorize) messages received during a disaster response. 

This notebook contains code to perform ETL on raw message data to be used for training. It consists of 2 parts:
1. Interactive development of an ETL pipeline
2. Code cells to test and generate a Python script to perform the ETL developed in the first part in production. *Python script generation is done using [nbdev](https://github.com/fastai/nbdev)*.

## Interactive Development

### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [ ]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
import os

In [ ]:
# Change directory to the data directory, relative to the directory containing this .ipynb
os.chdir('data')

In [ ]:
# load messages dataset
messages = pd.read_csv("messages.csv")
messages.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [ ]:
messages.genre.value_counts()

news      13068
direct    10782
social     2398
Name: genre, dtype: int64

In [ ]:
# load categories dataset
categories = pd.read_csv("categories.csv")
categories.head()

,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


### 2. Merge datasets.
- Merge the messages and categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

Before merging, are the ids in each dataset unique?

In [ ]:
def dup_counts(df, cols):
    """Find duplicate rows in `df`, considering the columns with names in the list 'cols`.
    
    Return:
     - A dataframe containing the values of `cols` for which there are multiple rows, and 
     the multiple row count in column 'count'.
    """
    df_gb = df.groupby(cols).size().reset_index().rename(columns={0:'count'})
    df_dups = df_gb[df_gb['count'] > 1]
    return df_dups 

In [ ]:
(dup_counts(messages, 'id').shape[0],
 dup_counts(messages, ['id', 'message']).shape[0],
 dup_counts(messages, ['id', 'message', 'genre']).shape[0])

(67, 67, 67)

In [ ]:
dup_counts(messages, ['id', 'message', 'original', 'genre']).shape[0]

31

In `messages`, there are 67 rows with the same values for columns 'id', 'message', 'genre'. The number of duplicates drops to 31 if we also consider the `original` column.

In [ ]:
# Check for duplicates in categories
dup_counts(categories, 'id').shape[0]

67

In [ ]:
dup_counts(categories, list(categories.columns)).shape[0]

31

In [ ]:
categories.shape[0], messages.shape[0]

(26248, 26248)

Since the duplicate counts for `categories` matches those for `messages`, it appears that these two dataframes were split from one another for the purpose of this exercise. 

We have no way of knowing which of the categorizations for a given message are more accurate, so for now we will retain all this data. After the join of `messages` and `categories`, we can remove any duplicates. This will still allow the possibility that the training data will have multiple training instances for a given message, with differing categorizations. Each of the provided classifications will contribute to the training.

If it were possible to redesign the categorization process, it would make sense to ensure that the `messages` data does not contain duplicates before generating categorizations.

-------------

In [ ]:
# merge datasets
df = messages.merge(categories, on='id')
df.head()

,id,message,original,genre,categories
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,related-1;request-0;offer-0;aid_related-0;medi...


### 3. Split `categories` into separate category columns.
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. You'll find [this method](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.str.split.html) very helpful! Make sure to set `expand=True`.
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `categories` with new column names.

In [ ]:
# create a dataframe of the 36 individual category columns
categories = df.categories.str.split(';', expand=True)
categories.head()

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


In [ ]:
# select the first row of the categories dataframe
row = categories.iloc[0]

# use this row to extract a list of new column names for categories.
category_colnames = [s[:-2] for s in row]
print(category_colnames)

['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']


In [ ]:
# rename the columns of `categories`
categories.columns = category_colnames
categories.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


### 4. Convert category values to just numbers 0 or 1.
- Iterate through the category columns in df to keep only the last character of each string (the 1 or 0). For example, `related-0` becomes `0`, `related-1` becomes `1`. Convert the string to a numeric value.
- You can perform [normal string actions on Pandas Series](https://pandas.pydata.org/pandas-docs/stable/text.html#indexing-with-str), like indexing, by including `.str` after the Series. 

In [ ]:
for column in categories:
    # set each value to be the last character of the string
    categories[column] = categories[column].str[-1]
    
    # convert column from string to numeric
    categories[column] = categories[column].astype(int)
categories.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 5. Replace `categories` column in `df` with new category columns.
- Drop the categories column from the df dataframe since it is no longer needed.
- Concatenate df and categories data frames.

In [ ]:
# drop the original categories column from `df`
df = df.drop('categories', axis=1)

df.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [ ]:
# concatenate the original dataframe with the new `categories` dataframe
df = df.join(categories)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 6. Remove duplicates.
- Check how many duplicates are in this dataset
- Drop the duplicates.
- Confirm duplicates were removed.

In [ ]:
# check number of duplicates


In [ ]:
def n_dup_rows(df, cols):
    """Return the number of rows in `df` that are duplicates, when considering columns `cols`."""
    return dup_counts(df, cols)['count'].sum()

In [ ]:
df_cols = list(df.columns)
print(df_cols)

['id', 'message', 'original', 'genre', 'related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']


In [ ]:
# Number of duplicate rows, considering all columns
n_dup_rows(df, df_cols)

124

In [ ]:
# Number of duplicate values in column 'id'
n_dup_rows(df, ['id'])

273

In [ ]:
# Number of duplicate values in column 'message'
n_dup_rows(df, ['message'])

277

In [ ]:
# number of duplicate rows, when not considering columns used in learning: ['id', 'original', 'genre']
subset_cols = df_cols
subset_cols.remove('id')
subset_cols.remove('original')
subset_cols.remove('genre')
n_dup_rows(df, subset_cols)

275

**Drop Duplicates**

The input to our classification is the 'message' column, and the 'original' column will not be used as an input to the classification. Exclude the 'original' column from the columns for consideration as duplicates so that when we drop duplicates for `df`, we will have multiple rows for a message only if the categorizations differ. (If we considered two or more rows as non-duplicate if they differed only by the 'original' column, that would amount to having multiple identical training instances, and there is no good reason for this overweighting of this instance.)

In [ ]:
# drop duplicates
df = df.drop_duplicates()

In [ ]:
# check number of duplicates
n_dup_rows(df, list(df.columns))

0

In [ ]:
# Number of duplicate rows, considering only columns used in learning
n_dup_rows(df, subset_cols)

2

In [ ]:
dup_counts(df, subset_cols)

,message,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,...,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report,count
175,#NAME?,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2


In [ ]:
df.shape

(26216, 40)

In [ ]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 7. Save the clean dataset into an sqlite database.
You can do this with pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLAlchemy library. Remember to import SQLAlchemy's `create_engine` in the first cell of this notebook to use it below.

In [ ]:
engine = create_engine('sqlite:///CleanedMessages.db')
df.to_sql('CategorizedMessages', engine, index=False, if_exists='replace')

### 8. Use this notebook to complete `etl_pipeline.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database based on new datasets specified by the user. Alternatively, you can complete `etl_pipeline.py` in the classroom on the `Project Workspace IDE` coming later.

In [ ]:
!ls data

CleanedMessages.db
categories.csv
messages.csv
test_save.db


## ETL Script for Production `process_data.py`
Develop and test code here; export to `process_data.py` using nbdev.

In [ ]:
# default_exp process_data

In [ ]:
#hide

# Do this to see possible %nbdev_ magics
from nbdev import *

from fastcore.test import *

In [ ]:
#export

import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import sys

In [ ]:
#export

def load_data(messages_filepath, categories_filepath):
    """Load message and category data from the specified filepaths.
    
    Args:
     - messages_filepath (str): path to messages csv file
     - categories_filepath (str): path to categories csv file
     
    Returns:
     - DataFrame of the two datasets, joined on 'id'.
    """
    messages = pd.read_csv(messages_filepath)
    categories = pd.read_csv(categories_filepath)
    return messages.merge(categories, on='id')

Example call to `load_data()`:

In [ ]:
loaded = load_data("messages.csv", "categories.csv")
loaded.head()

,id,message,original,genre,categories
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,related-1;request-0;offer-0;aid_related-0;medi...


In [ ]:
#export

def clean_data(df):
    """Clean the loaded messages and categories data.
     Convert 'categories' column string into a set of numeric columns, and remove duplicate rows.
     
     Arg:
      - df: DataFrame containing data loaded by load_data()
      
     Returns:
      - DataFrame of cleaned data.
     """
    # Categories are in a single string column; create a dataframe with a column for each category
    categories = df.categories.str.split(';', expand=True)
    
    # select the first row of the categories dataframe
    row = categories.iloc[0]
    # use this row to extract a list of new column names for categories.
    category_colnames = [s[:-2] for s in row]
    # rename the columns of `categories`
    categories.columns = category_colnames
    
    # convert category values to numbers
    for column in categories:
        # set each value to be the last character of the string
        categories[column] = categories[column].str[-1]

        # convert column from string to numeric
        categories[column] = categories[column].astype(int)
    
    # replace original category string values with numeric columns
    df = df.drop('categories', axis=1)
    
    # concatenate the original dataframe with the new `categories` dataframe
    df = df.join(categories)
    
    # remove duplicates
    return df.drop_duplicates()

Example call to `clean_data()`

In [ ]:
cleaned = clean_data(loaded)
cleaned.head(3)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
expected_cols = ['id', 'message', 'original', 'genre', 'related', 'request', 'offer', 'aid_related', 
                 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 
                 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 
                 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 
                 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 
                 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 
                 'direct_report']
assert expected_cols == list(cleaned.columns)

In [ ]:
#hide

# Tests to validate types of columns

assert cleaned.dtypes['id'] == np.dtype('int64')

assert type(cleaned.iloc[0].loc['message']) == str
assert type(cleaned.iloc[0].loc['original']) == str
assert type(cleaned.iloc[0].loc['genre']) == str

# verify we have ints for all the category columns
for _, categ_type in cleaned.dtypes.loc['related':].iteritems():
    assert np.issubdtype(categ_type, np.integer)

In [ ]:
#export

def get_engine(database_filename):
    """Return database engine given database filename."""
    return create_engine(f'sqlite:///{database_filename}')

def save_data(df, engine):
    """Save the specified DataFrame as table 'CategorizedMessages' to specified database `engine`."""
    with engine.connect() as connection:
        df.to_sql('CategorizedMessages', connection, index=False, if_exists='replace')

Example calls to `get_engine()`/`save_data()`

In [ ]:
# small datframe to test saving
test_df = cleaned.iloc[:5]

# save the dataframe
engn = get_engine("test_save.db")
save_data(test_df, engn)

In [ ]:
#hide
# Test reading the dataframe back in again
with engn.connect() as conn:
    written_test = pd.read_sql('CategorizedMessages', conn)

written_test.head(3)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#hide
# Verify that the re-read dataframe looks correct
assert list(written_test.columns) == expected_cols
assert written_test.shape == (5, 40)

---

In [ ]:
# exporti

def main():
    if len(sys.argv) == 4:

        messages_filepath, categories_filepath, database_filepath = sys.argv[1:]

        print('Loading data...\n    MESSAGES: {}\n    CATEGORIES: {}'
              .format(messages_filepath, categories_filepath))
        df = load_data(messages_filepath, categories_filepath)

        print('Cleaning data...')
        df = clean_data(df)
        
        print('Saving data...\n    DATABASE: {}'.format(database_filepath))
        engine = get_engine(database_filepath)
        save_data(df, engine)
        
        print('Cleaned data saved to database!')
    
    else:
        print('Please provide the filepaths of the messages and categories '\
              'datasets as the first and second argument respectively, as '\
              'well as the filepath of the database to save the cleaned data '\
              'to as the third argument. \n\nExample: python process_data.py '\
              'disaster_messages.csv disaster_categories.csv '\
              'DisasterResponse.db')

        
# Don't run main() when this cell is run in a notebook; use try so exported module
# has no dependency on nbdev
try: from nbdev.imports import IN_NOTEBOOK
except: IN_NOTEBOOK=False

if __name__ == '__main__' and not IN_NOTEBOOK:
    main()